In [ ]:
suppressPackageStartupMessages({
    source(".Rprofile")
    library(dplyr)
    library(cicero)
})

In [ ]:
PATH <- "e01_cicero"
if (!dir.exists(PATH)) {
    dir.create(PATH)
}

# Read data

In [ ]:
atac <- read_h5ad("s01_preprocessing/atac.h5ad")

In [ ]:
cellinfo <- atac$obs
head(cellinfo, n = 2)

In [ ]:
peakinfo <- atac$var %>% select(
    chr = chrom, bp1 = chromStart, bp2 = chromEnd
) %>% mutate(
    site_name = paste(chr, bp1, bp2, sep = "_")
)
rownames(peakinfo) <- peakinfo$site_name
head(peakinfo, n = 2)

In [ ]:
indata <- t(atac$X)
indata@x[indata@x > 0] <- 1
rownames(indata) <- rownames(peakinfo)
colnames(indata) <- rownames(cellinfo)

In [ ]:
input_cds <- newCellDataSet(
    indata,
    phenoData = methods::new("AnnotatedDataFrame", data = cellinfo),
    featureData = methods::new("AnnotatedDataFrame", data = peakinfo),
    expressionFamily = VGAM::binomialff(),
    lowerDetectionLimit = 0
)

# Cicero

In [ ]:
set.seed(2021)
input_cds <- detectGenes(input_cds)
input_cds <- estimateSizeFactors(input_cds)
input_cds <- reduceDimension(
    input_cds, max_components = 2, num_dim = 6,
    reduction_method = "tSNE", norm_method = "none"
)

In [ ]:
tsne_coords <- t(reducedDimA(input_cds))
row.names(tsne_coords) <- row.names(pData(input_cds))
cicero_cds <- make_cicero_cds(input_cds, reduced_coordinates = tsne_coords)

In [ ]:
human.hg38.genome <- read.table("../../data/genome/GRCh38.p12.genome.fa.fai", stringsAsFactors = TRUE)[, 1:2]

In [ ]:
conns <- run_cicero(cicero_cds, human.hg38.genome, window=3e5)
head(conns)

In [ ]:
write.csv(conns, gzfile(file.path(PATH, "cicero_conns.csv.gz")), row.names = FALSE, quote = FALSE)